Start



In [4]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import some libraries

In [5]:

import os
import sys
import lmdb
import pickle
import numpy as np

# Cloning from github (Generative Models)



In [3]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach



Cloning into 'Generative_Models'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 15891 (delta 70), reused 91 (delta 32), pack-reused 15727
Receiving objects: 100% (15891/15891), 733.49 MiB | 40.58 MiB/s, done.
Resolving deltas: 100% (10817/10817), done.
Checking out files: 100% (1697/1697), done.


# Cloning from github (LMDB_Datasets)

In [6]:
!cd Generative_Models
!git clone https://github.com/azeghost/LMDB_Datasets # this is for loading git with correct brach

Cloning into 'LMDB_Datasets'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 257 (delta 19), reused 20 (delta 7), pack-reused 214
Receiving objects: 100% (257/257), 407.92 MiB | 45.81 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [11]:
!ls data/.CBSD68/01

0000.png  0009.png  0018.png  0027.png	0036.png  0045.png  0054.png  0063.png
0001.png  0010.png  0019.png  0028.png	0037.png  0046.png  0055.png  0064.png
0002.png  0011.png  0020.png  0029.png	0038.png  0047.png  0056.png  0065.png
0003.png  0012.png  0021.png  0030.png	0039.png  0048.png  0057.png  0066.png
0004.png  0013.png  0022.png  0031.png	0040.png  0049.png  0058.png  0067.png
0005.png  0014.png  0023.png  0032.png	0041.png  0050.png  0059.png
0006.png  0015.png  0024.png  0033.png	0042.png  0051.png  0060.png
0007.png  0016.png  0025.png  0034.png	0043.png  0052.png  0061.png
0008.png  0017.png  0026.png  0035.png	0044.png  0053.png  0062.png


In [ ]:
!ls Generative_Models/data/.CBSD68/

01  CBSD68.tar.bz


# LMDB transformer

In [12]:
!pip install colorlog

In [13]:
from colorlog import ColoredFormatter




**LMDB Transformer **

In [ ]:
!cp -r LMDB_Datasets/transformation/ Generative_Models/

In [9]:
import os
os.chdir('Generative_Models')
os.getcwd()

'/content/Generative_Models'

In [14]:
import tensorflow as tf

In [15]:
image_size_original=(481, 321, 3)
scale = 3
image_size = list(map(lambda x: x//scale , image_size_original[:-1])) + [image_size_original[-1]]
image_size = (*image_size,)

In [16]:
inputs_shape = list(map(lambda x: x//2 , image_size[:-1])) + [image_size[-1]]
inputs_shape = (*inputs_shape,)

In [115]:
shrink_fn = lambda image: tf.image.resize(image, inputs_shape[:-1]).numpy()

In [118]:
from LMDB_Datasets.transformation.lmdb_transformer import SRLmdbTransformer

In [63]:
!ls 

data		   graphs	  notebooks	    statistical     utils
evaluation	   LICENSE	  README.md	    training
Generative_Models  LMDB_Datasets  requirements.txt  transformation


In [119]:
images_dir = 'data/.CBSD68' #Folder to images ( without name of the folder which we moved images before)
validation_percentage = 30
valid_format = 'png'

In [120]:
transformer = SRLmdbTransformer(image_dir = images_dir, trans_func=shrink_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [121]:

val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

47 21


# Getting labels by filename

In [122]:
from LMDB_Datasets.datasets_customed.label_by_filename import get_label_by_filename


In [123]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [124]:
lmdb_dir = 'data/.CBSD68_low2high_LMDB'
lmdb_dir

'data/.CBSD68_low2high_LMDB'

In [70]:
#!mv data/.CBSD68/02 data/.CBSD68/a

In [71]:
!ls data/.CBSD68/a

ls: cannot access 'data/.CBSD68/a': No such file or directory


In [125]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = lmdb_dir
           ,category='training',target_size=(481, 321),color_mode='rgb')


Total number of imgs for catagory 47
Storing 47data/.CBSD68_low2high_LMDB/into _training from folder 01


In [126]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = lmdb_dir
           ,category='validation',target_size=(481, 321),color_mode='rgb')

Total number of imgs for catagory 21
Storing 21data/.CBSD68_low2high_LMDB/into _validation from folder 01


# Saving meta data of dataset

In [131]:
!touch ../Generative_Models/data/.CBSD68_low2high_LMDB/meta_info.json #creating a json file

In [130]:
!ls ../Generative_Models/data/.CBSD68_low2high_LMDB

_training  _validation


In [133]:

info = {
    'Dataset_name': "CBSD68",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata('../Generative_Models/data/.CBSD68_low2high_LMDB',info)

In [134]:
transformer.get_metadata('../Generative_Models/data/.CBSD68_low2high_LMDB') #checking whether it is working or not

{'Dataset_name': 'CBSD68',
 'Dataset_size': 68,
 'tra_num_images': 47,
 'val_num_images': 21}

In [136]:
!ls -l /content/Generative_Models/data/.CBSD68_low2high_LMDB

ls: cannot access '/content/Generative_Models/data/.CBSD68_LMDB/': No such file or directory


# I tried to read LMDB, but..

In [ ]:
import lmdb
import pickle

def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

            for label in labels_list:
                # _lab = {label: eval(f'dataset.{label}')}
#                 print({label: eval(f'dataset.{label}')})
                # labels = {**labels, **_lab}
                if label in labels:
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels = {label: [eval(f'dataset.{label}')] }
                
    env.close()
    return {'images': images, **labels}

I don't know why it gives the following error.

In [ ]:
ds = read_many_lmdb('/content/Generative_Models/CBSD68_LMDB/_training', 5)

TypeError: ignored

# Zipping

In [153]:
!ls data/.CBSD68_low2high_LMDB/_training/

data.mdb  lock.mdb


In [137]:
!tar -cvjf CBSD68_low2high_LMDB.tar.bz data/.CBSD68_low2high_LMDB

data/.CBSD68_low2high_LMDB/
data/.CBSD68_low2high_LMDB/_training/
data/.CBSD68_low2high_LMDB/_training/data.mdb
data/.CBSD68_low2high_LMDB/_training/lock.mdb
data/.CBSD68_low2high_LMDB/meta_info.json
data/.CBSD68_low2high_LMDB/_validation/
data/.CBSD68_low2high_LMDB/_validation/data.mdb
data/.CBSD68_low2high_LMDB/_validation/lock.mdb


In [138]:
!ls CBSD68_low2high_LMDB.tar.bz -l

-rw-r--r-- 1 root root 21076016 Oct 31 10:06 CBSD68_low2high_LMDB.tar.bz


In [148]:
!zip -r LMDB_Datasets.zip  LMDB_Datasets/

updating: LMDB_Datasets/ (stored 0%)
updating: LMDB_Datasets/.git/ (stored 0%)
updating: LMDB_Datasets/.git/info/ (stored 0%)
updating: LMDB_Datasets/.git/info/exclude (deflated 28%)
updating: LMDB_Datasets/.git/packed-refs (deflated 10%)
updating: LMDB_Datasets/.git/logs/ (stored 0%)
updating: LMDB_Datasets/.git/logs/HEAD (deflated 26%)
updating: LMDB_Datasets/.git/logs/refs/ (stored 0%)
updating: LMDB_Datasets/.git/logs/refs/remotes/ (stored 0%)
updating: LMDB_Datasets/.git/logs/refs/remotes/origin/ (stored 0%)
updating: LMDB_Datasets/.git/logs/refs/remotes/origin/HEAD (deflated 26%)
updating: LMDB_Datasets/.git/logs/refs/heads/ (stored 0%)
updating: LMDB_Datasets/.git/logs/refs/heads/master (deflated 26%)
updating: LMDB_Datasets/.git/HEAD (stored 0%)
updating: LMDB_Datasets/.git/objects/ (stored 0%)
updating: LMDB_Datasets/.git/objects/pack/ (stored 0%)
updating: LMDB_Datasets/.git/objects/pack/pack-18fc8702a9020dd2ee622281f7f16a0e09d022a1.pack (deflated 0%)
updating: LMDB_Datasets/

In [142]:
!ls

CBSD68_low2high_LMDB.tar.bz  graphs	    README.md	      transformation
data			     LICENSE	    requirements.txt  utils
evaluation		     LMDB_Datasets  statistical
Generative_Models	     notebooks	    training


In [149]:
!tar -cvjf LMDB_Datasets.tar.bz LMDB_Datasets

LMDB_Datasets/
LMDB_Datasets/.git/
LMDB_Datasets/.git/info/
LMDB_Datasets/.git/info/exclude
LMDB_Datasets/.git/packed-refs
LMDB_Datasets/.git/logs/
LMDB_Datasets/.git/logs/HEAD
LMDB_Datasets/.git/logs/refs/
LMDB_Datasets/.git/logs/refs/remotes/
LMDB_Datasets/.git/logs/refs/remotes/origin/
LMDB_Datasets/.git/logs/refs/remotes/origin/HEAD
LMDB_Datasets/.git/logs/refs/heads/
LMDB_Datasets/.git/logs/refs/heads/master
LMDB_Datasets/.git/HEAD
LMDB_Datasets/.git/objects/
LMDB_Datasets/.git/objects/pack/
LMDB_Datasets/.git/objects/pack/pack-18fc8702a9020dd2ee622281f7f16a0e09d022a1.pack
LMDB_Datasets/.git/objects/pack/pack-18fc8702a9020dd2ee622281f7f16a0e09d022a1.idx
LMDB_Datasets/.git/objects/info/
LMDB_Datasets/.git/hooks/
LMDB_Datasets/.git/hooks/pre-applypatch.sample
LMDB_Datasets/.git/hooks/commit-msg.sample
LMDB_Datasets/.git/hooks/prepare-commit-msg.sample
LMDB_Datasets/.git/hooks/pre-commit.sample
LMDB_Datasets/.git/hooks/applypatch-msg.sample
LMDB_Datasets/.git/hooks/post-update.sample